# Modeling elections

In [9]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pystan

## Data
The electoral_votes variable is a dictionary containing the number of Electoral College votes for each state. For example

  >>> electoral_votes['Indiana']
  11
Data from Wikipedia: United_States_Electoral_College

The survey_results variable is a dictionary mapping from states to an array of survey results for each candidate. Each row in a survey results array represents one survey and each column represents one candidate. There are 4 columns, representing Clinton, Trump, Johnson, and Stein in that order. In the example below, Clinton got 340 votes in the first survey, Trump got 258, Johnson got 27, and Stein got 13.

  >>> survey_results['Indiana']
  array([[340, 258,  27,  13],
         [240, 155,   5,   5],
         [235, 155,  50,  20],
         [308, 266,  49,  35],
         [222, 161,  80,  30]])
Data from Wikipedia: Statewide opinion polling for the United States presidential election, 2016

In [51]:
electoral_votes = {
    'Alabama': 9,
    'Alaska': 3,
    'Arizona': 11,
    'Arkansas': 6,
    'Colorado': 9,
}

survey_results = {
    'num_sur': 4,
    'num_cand': 4,
    'Alabama': np.array([], dtype=int).reshape(0, 4),
    'Alaska': np.array([400 * np.array([.47, .43, .07, .03]), 500 * np.array([.36, .37, .07, .03]), 500 * np.array([.34, .37, .10, .02]), 660 * np.array([.31, .36, .18, .06])], dtype=int),
    'Arizona': np.array([392 * np.array([.45, .47, .05, .02]), 550 * np.array([.39, .47, .04, .03]), 719 * np.array([.40, .45, .09, .03]), 769 * np.array([.44, .49, .05, .01]), 2229 * np.array([.45, .44, .07, .01]), 700 * np.array([.43, .47, .02, .02]), 550 * np.array([.41, .45, .03, .01]), 994 * np.array([.42, .44, .04, .01]), 550 * np.array([.40, .42, .05, .02]), 2385 * np.array([.48, .46, .05, .01]), 401 * np.array([.45, .46, .04, .01]), 550 * np.array([.41, .41, .05, .02]), 1538 * np.array([.39, .44, .06, .02]), 713 * np.array([.43, .38, .06, .01]), 400 * np.array([.39, .37, .08, .03]), 600 * np.array([.44, .42, .09, .01]), 718 * np.array([.42, .42, .05, .01]), 484 * np.array([.41, .46, .09, .01]), 649 * np.array([.38, .40, .12, .03])], dtype=int),
    'Arkansas': np.array([463 * np.array([.33, .56, .04, .02]), 831 * np.array([.34, .55, .03, .01]), 600 * np.array([.29, .57, .05, .03])], dtype=int),
    'Colorado': np.array([1150 * np.array([.45, .44, .05, .04]), 500 * np.array([.44, .38, .07, .02]), 550 * np.array([.39, .39, .05, .04]), 750 * np.array([.44, .41, .08, .04]), 685 * np.array([.45, .37, .10, .03]), 400 * np.array([.49, .38, .07, .03]), 602 * np.array([.44, .33, .10, .03]), 694 * np.array([.46, .40, .06, .02]), 784 * np.array([.41, .42, .13, .03]), 991 * np.array([.40, .39, .07, .02]), 644 * np.array([.44, .42, .10, .02]), 540 * np.array([.41, .34, .12, .03]), 600 * np.array([.38, .42, .13, .02]), 704 * np.array([.48, .43, .04, .02]), 605 * np.array([.43, .38, .07, .02]), 997 * np.array([.42, .39, .07, .02])], dtype=int),
}

states = sorted(survey_results.keys())
# print('Modeling', len(states), 'states with', sum(electoral_votes[s] for s in states), 'electoral college votes')
# print(survey_results['Alaska'])

## Generative model
For each state we generate an $\vec{\alpha}$ vector, which defines a Dirichlet distribution over the proportion of votes that go to each of the 4 candidates whenever we do a survey — including the final survey, namely the election itself which we want to predict. The prior over each component of $\vec{\alpha}$ is taken as a Cauchy distribution with location 0 and scale 1. Since the components of $\vec{\alpha}$ are positive, we actually use the positive half-Cauchy distribution.

For each survey in a state we generate a probability vector $\vec{p_i} \sim \text{Dirichlet}(\vec{\alpha})$ for the probability that a voter selects each of the 4 candidates.

For each survey, we then generate the number of votes going to each candidate as $\vec{k_i} \sim \text{Multinomial}(\vec{p_i})$.

## Tasks
Use Stan to sample from the posterior distribution over $\alpha$ and visualize your results. There are 10 states, so you will have 10 posteriors.
The posteriors over $\alpha$ show a lot of variation between different states. Explain the results you get in terms of the model and the data.

In [30]:
stan_data = {
    'Alabama': {
        'data_set': survey_results['Alabama'],
        'num_sur': 4,
        'num_cand': 4,
    },
    'Alaska': {
        'data_set': survey_results['Alaska'],
        'num_sur': 4,
        'num_cand': 4,
    },
    'Arizona': {
        'data_set': survey_results['Arizona'],
        'num_sur': 4,
        'num_cand': 4,
    },
    'Arkansas': {
        'data_set': survey_results['Arkansas'],
        'num_sur': 4,
        'num_cand': 4,
    },
    'Colorado': {
        'data_set': survey_results['Colorado'],
        'num_sur': 4,
        'num_cand': 4,
    }
}

In [31]:
stan_code = '''

data {
    int<lower=0> num_sur;
    int<lower=0> num_cand;
    int<lower=0> data_set[num_sur, num_cand];
}

parameters {
    vector<lower=0>[4] alpha;
    simplex[4] p;
    
}

model {
    alpha ~ cauchy(0, 1);
    for (i in 1:num_sur) {
        p ~ dirichlet(alpha);
        data_set[i] ~ multinomial(p);
    }
}
'''

stan_model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2e92d9b2cee72e4ae4601d7470d6c894 NOW.


In [37]:
results_alaska = stan_model.sampling(data=stan_data['Alaska'])
print(results_alaska)

Inference for Stan model: anon_model_2e92d9b2cee72e4ae4601d7470d6c894.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[1]  1.5e7   1.8e7  2.6e7 2207.6  7.9e4  3.4e5  1.1e7  7.6e7      2   3.84
alpha[2]  1.6e7   1.9e7  2.8e7 2267.5  8.2e4  3.6e5  1.1e7  8.1e7      2   3.83
alpha[3]  4.8e6   5.7e6  8.5e6 656.64  2.4e4  1.1e5  3.5e6  2.4e7      2   3.84
alpha[4]  1.2e6   1.5e6  2.2e6 249.06 8095.9  3.4e4  9.2e5  6.3e6      2   3.98
p[1]       0.41  1.2e-3 8.2e-3   0.39    0.4   0.41   0.41   0.42     43   1.09
p[2]       0.43  1.5e-3 9.1e-3   0.41   0.42   0.43   0.43   0.44     38   1.14
p[3]       0.13  1.4e-3 6.6e-3   0.11   0.12   0.13   0.13   0.14     22    1.2
p[4]       0.04  2.9e-3 5.1e-3   0.03   0.04   0.04   0.04   0.05      3   2.05
lp__      -2024    3.88   6.36  -2036  -2028  -2025  -2019  -2013      3   2.39

Samples we

In [33]:
results_alaska.stansummary(pars=['alpha'], probs=[0.025, 0.5, 0.975])

'Inference for Stan model: anon_model_2e92d9b2cee72e4ae4601d7470d6c894.\n4 chains, each with iter=2000; warmup=1000; thin=1; \npost-warmup draws per chain=1000, total post-warmup draws=4000.\n\n           mean se_mean     sd   2.5%    50%  97.5%  n_eff   Rhat\nalpha[1]  2.0e7   1.5e7  2.5e7  1.2e6  9.9e6  9.6e7      3   3.25\nalpha[2]  2.2e7   1.7e7  2.8e7  1.3e6  1.1e7  1.0e8      3   3.49\nalpha[3]  7.4e6   5.7e6  9.1e6  4.1e5  3.4e6  3.4e7      2   3.54\nalpha[4]  2.0e6   1.6e6  2.5e6  1.3e5  9.3e5  9.9e6      3    3.3\n\nSamples were drawn using NUTS at Thu Oct 24 13:22:06 2019.\nFor each parameter, n_eff is a crude measure of effective sample size,\nand Rhat is the potential scale reduction factor on split chains (at \nconvergence, Rhat=1).'

In [ ]:
def abc(alaska):
    results_alaska = stan_model.sampling(data=stan_data[alaska])
    results_alaska.stansummary(pars=['alpha'], probs=[0.025, 0.5, 0.975])
    samples_alaska = results_alaska.extract()
    p_predicted = np.empty(samples_alaska['alpha'].shape)
    for i in range(samples['alpha'].shape[0]):
        p_predicted[i] = stats.dirichlet(samples['alpha'][i]).rvs()
    alaska = np.mean(p_predicted, axis=0)
    return alaska

## Simulation time
Use the posterior samples to predict the outcome of the presidential elections.

Predict the probability that each candidate will win each state.
Use the posterior $\alpha$ samples to generate posterior predictive samples for $p$ — the proportion of votes each candidate would get in each state in an election.
Use these $p$ samples to estimate the probability that each candidate will win each state.
Predict the probability that each candidate will win the presidential election.
Use the posterior predictive probability that each candidate will win each state to generate samples over the total number Electoral College votes each candidate would get in an election.
Use the total number of votes to generate samples over who would win the election.

In [50]:
final = {'clinton': 0, 'trump': 0, 'johnson': 0, 'stein': 0}
candidates = ['clinton', 'trump', 'johnson', 'stein']

samples_alaska = results_alaska.extract()
# Make a new array with same dimensions as alpha
p_predicted = np.empty(samples_alaska['alpha'].shape)
# Generate one p sample for each alpha sample
for i in range(samples['alpha'].shape[0]):
    p_predicted[i] = stats.dirichlet(samples['alpha'][i]).rvs()
alaska = np.mean(p_predicted, axis=0)
max_alaska = np.max(alaska)
ind = np.where(alaska == max_alaska)[0]
final[candidates[ind[0]]] += electoral_votes['Alaska']

In [ ]:
for i in range(5):
    s